# Natural Orbital Functionals: Method Comparison at 0.02 and 0.002 step size

Import libreries

In [1]:
from scipy.linalg import eigh
import pynof
import numpy as np
import csv
import sys
import time as t
# Define energy.py path, in our case is in myCode dir, two directories above
sys.path.insert(1, '../../myCode/')
# read files with optimization functions
from energy import *

Create molecule and choose basis set

In [ ]:
# Molecules definition
#Time for CO2: 457.8015 s
#Time for CO: 379.3235 s
#Time for H2O: 353.4587 s
#Total Time: 1190.5839891433716

co = pynof.molecule("""
0 1
  C      0.0       0.0         0.0
  O      0.0       0.0         1.12     
""")

co2 = pynof.molecule("""
0 1
C    0.0000    0.0000    0.0000   
O    1.1000    0.0000    0.0000   
O    -1.1000   -0.0000    0.0000 
""")

h2o = pynof.molecule("""
0 1
  O  0.0000   0.000   0.116
  H  0.0000   0.749  -0.453
  H  0.0000  -0.749  -0.453
""")

# Dictionary to facilitate molecule selection with 
# output file name

molecules = {
    "CO": co,
    "CO2": co2,
    "H2O": h2o,
}

Run NOF calculations with several optimization algorithms at 0.002 step size.

In [ ]:
times = {} 
#molecule_list = ["CO2","CO","H2O"]
molecule_list = ["CO","H2O"]

alpha_list=[0.02,0.002]

t_st=t.time()
for molecule in molecule_list:
    st=t.time()
    for alpha in alpha_list:  
        mol = molecules[molecule]

        # Select basis set
        basis = "cc-pvdz"

        p = pynof.param(mol,basis)

        # Here we select the NOF functional.
        p.maxit = 25
        p.alpha = alpha

        algorithms = [ 'SD', 'RMSPROP', 'ADAM' ]

        # File name to store the data
        filename = f"{molecule}_{p.alpha}.csv"

        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["alg", "iteration", "energy"])  # Header

            for alg in algorithms:
                p.orb_method=alg
                _, energy_data = calc_nof_orbrot(mol, p)
                for i_ext, energy in energy_data:
                    writer.writerow([alg, i_ext, energy])
    et=t.time()
    times[molecule] = et - st
t_et=t.time()  

for molecule in molecule_list:
     print(f"Time for {molecule}: {times[molecule]:.4f} s")

print(f"Total Time:",t_et-t_st)

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry C          line   138 file /home/pobmelat/miniconda3/envs/pynof2026/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry O          line   198 file /home/pobmelat/miniconda3/envs/pynof2026/share/psi4/basis/cc-pvdz.gbs 

Number of basis functions                   (NBF)    = 42
Inactive Doubly occupied orbitals up too     (NO1)    = 0
No. considered Strongly Doubly occupied MOs (NDOC)   = 11
No. considered Strongly Singly occupied MOs (NSOC)   = 0
No. of Weakly occ. per St. Doubly occ.  MOs (NCWO)   = 2
Dimension of the Nat. Orb. subspace         (NBF5)   = 33
No. of electrons                                     = 22
Multiplicity                                         = 1

No violations of the orthonormality

PNOF8 Calculation (SD/Softmax Optimization)

Nitext  Nit_orb  Nit_occ      Eelec           Etot          Ediff        Grad_orb   Grad_occ Conv Orb Conv Occ
alpha =  0.02
Starting SD